## Basic Example

### First define the signature

In [21]:
import dspy 
from open_intent_classifier.utils import join_labels

class Classification(dspy.Signature):
    """Classify the customer message into one of the intent labels.
    The output should be only the predicted class as a single intent label."""

    customer_message = dspy.InputField(desc="Customer message during customer service interaction")
    intent_labels = dspy.InputField(desc="Labels that represent customer intent")
    intent_class = dspy.OutputField(desc="a label best matching customer's intent ")

### Define the languae model 

In [24]:
lm_mini = dspy.OpenAI(model='gpt-4o-mini')
dspy.settings.configure(lm=lm_mini)

### Predict

In [25]:
predict = dspy.Predict(Classification)
labels = join_labels(["Cancel subscription", "Refund request"])
labels = " ".join(["Cancel subscription", "Refund request"])
text = "I want to cancel my subscription"
pred = predict(customer_message=text, intent_labels=labels)
pred

Prediction(
    intent_class='Cancel subscription'
)

In [26]:
lm_mini.inspect_history(n=1)




Classify the customer message into one of the intent labels.
The output should be only the predicted class as a single intent label.

---

Follow the following format.

Customer Message: Customer message during customer service interaction
Intent Labels: Labels that represent customer intent
Intent Class: a label best matching customer's intent

---

Customer Message: I want to cancel my subscription
Intent Labels: Cancel subscription Refund request
Intent Class: Cancel subscription





"\n\n\nClassify the customer message into one of the intent labels.\nThe output should be only the predicted class as a single intent label.\n\n---\n\nFollow the following format.\n\nCustomer Message: Customer message during customer service interaction\nIntent Labels: Labels that represent customer intent\nIntent Class: a label best matching customer's intent\n\n---\n\nCustomer Message: I want to cancel my subscription\nIntent Labels: Cancel subscription Refund request\nIntent Class:\x1b Cancel subscription\x1b\n\n\n"

## Using Chain Of Thought (CoT)

In [27]:
predict_with_chain_of_thought = dspy.ChainOfThought(Classification)
pred = predict_with_chain_of_thought(customer_message=text, intent_labels=labels)
pred

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Prediction(
    rationale='produce the intent_class. We first identify that the customer is expressing a desire to terminate an ongoing service, which directly aligns with the intent of canceling a subscription. There is no mention of seeking a refund or any other request, so the primary focus is on the cancellation itself.',
    intent_class='Cancel subscription'
)

In [28]:
pred.intent_class

'Cancel subscription'

In [29]:
pred.rationale

'produce the intent_class. We first identify that the customer is expressing a desire to terminate an ongoing service, which directly aligns with the intent of canceling a subscription. There is no mention of seeking a refund or any other request, so the primary focus is on the cancellation itself.'

In [30]:
lm_mini.inspect_history(n=1)




Classify the customer message into one of the intent labels.
The output should be only the predicted class as a single intent label.

---

Follow the following format.

Customer Message: Customer message during customer service interaction

Intent Labels: Labels that represent customer intent

Reasoning: Let's think step by step in order to ${produce the intent_class}. We ...

Intent Class: a label best matching customer's intent

---

Customer Message: I want to cancel my subscription

Intent Labels: Cancel subscription Refund request

Reasoning: Let's think step by step in order to produce the intent_class. We first identify that the customer is expressing a desire to terminate an ongoing service, which directly aligns with the intent of canceling a subscription. There is no mention of seeking a refund or any other request, so the primary focus is on the cancellation itself.

Intent Class: Cancel subscription





"\n\n\nClassify the customer message into one of the intent labels.\nThe output should be only the predicted class as a single intent label.\n\n---\n\nFollow the following format.\n\nCustomer Message: Customer message during customer service interaction\n\nIntent Labels: Labels that represent customer intent\n\nReasoning: Let's think step by step in order to ${produce the intent_class}. We ...\n\nIntent Class: a label best matching customer's intent\n\n---\n\nCustomer Message: I want to cancel my subscription\n\nIntent Labels: Cancel subscription Refund request\n\nReasoning: Let's think step by step in order to\x1b produce the intent_class. We first identify that the customer is expressing a desire to terminate an ongoing service, which directly aligns with the intent of canceling a subscription. There is no mention of seeking a refund or any other request, so the primary focus is on the cancellation itself.\n\nIntent Class: Cancel subscription\x1b\n\n\n"

## Example of a bug - not specifying output properly

### My instruction: 
"""Classify the customer message into one of the intent labels.""" 
### is not specific enough

In [31]:
import dspy 
from open_intent_classifier.utils import join_labels

class Classification(dspy.Signature):
    """Classify the customer message into one of the intent labels."""

    customer_message = dspy.InputField(desc="Customer message during customer service interaction")
    intent_labels = dspy.InputField(desc="Labels that represent customer intent")
    intent_class = dspy.OutputField(desc="a label best matching customer's intent ")

### Intent class is the whole message, instead of being the class

In [17]:
predict = dspy.Predict(Classification)
labels = " ".join(["Cancel subscription", "Refund request"])
text = "I want to cancel my subscription"
pred = predict(customer_message=text, intent_labels=labels)
pred

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Prediction(
    intent_class='Customer Message: I want to cancel my subscription  \nIntent Labels: Cancel subscription  \nIntent Class: Cancel subscription'
)